# 4: Creating and Evaluating Predictors - Related Time Series

This notebook will build off of all the earlier work and requires that at least the importing of target time series and related time series data be complete. If you have not performed those steps yet, go back, do so, then continue.

At this point, you now have a target-time-series dataset and a related-time-series dataset loaded into a singular Dataset Group, this is what is required to leverage the models that support related data in Amazon Forecast. If your data supports item-level metadata, it could be added to the dataset group as well and would benefit only DeepAR+. 

To continue the work, start with the imports, determine your region, establish your API connections, and load all previously stored values

In [2]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import uuid

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

In [ ]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

In [ ]:
%store -r

## Creating and Training Predictors
 
Given that our data is hourly and we want to generate a forecast on the hour, Forecast limits us to a horizon of 500 of whatever the slice is. This means we will be able to predict about 20 days into the future.

The cells below will define a few variables to be used with all of our models. Then there will be an API call to create each `Predictor` where they are based on Prophet and DeepAR+ respectfully.


In [ ]:
forecastHorizon = 480
NumberOfBacktestWindows = 4
BackTestWindowOffset = 480
ForecastFrequency = "H"

In [ ]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [ ]:
# Prophet Specifics
# Note the REL to indicate related time series data
prophet_predictorName = project+'_proph_rel'
print(prophet_predictorName)

In [ ]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
    PredictorName=prophet_predictorName, 
    AlgorithmArn=prophet_algorithmArn,
    ForecastHorizon=forecastHorizon,
    PerformAutoML= False,
    PerformHPO=False,
    EvaluationParameters={
        "NumberOfBacktestWindows": NumberOfBacktestWindows, 
        "BackTestWindowOffset": BackTestWindowOffset
    }, 
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn, 
        "SupplementaryFeatures": [ 
                     { 
                        "Name": "holiday",
                        "Value": "US"
                     }
                  ]
        },
    FeaturizationConfig={
        "ForecastFrequency": ForecastFrequency, 
        "Featurizations": [
                        {
                            "AttributeName": "target_value", 
                            "FeaturizationPipeline": [
                              {
                                  "FeaturizationMethodName": "filling", 
                                  "FeaturizationMethodParameters": 
                                  {
                                      "frontfill": "none", 
                                      "middlefill": "zero", 
                                      "backfill": "zero"
                                  }
                              }
                            ]
                          }
                        ]
    }
 )




### DeepAR+

In [ ]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deep_rel'

In [ ]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
    PredictorName=deeparp_predictorName, 
    AlgorithmArn=deepAR_Plus_algorithmArn,
    ForecastHorizon=forecastHorizon,
    PerformAutoML= False,
    PerformHPO=False,
    EvaluationParameters= {
        "NumberOfBacktestWindows": NumberOfBacktestWindows, 
        "BackTestWindowOffset": BackTestWindowOffset
    }, 
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn, 
        "SupplementaryFeatures": [ 
                     { 
                        "Name": "holiday",
                        "Value": "US"
                     }
                  ]
    },
    FeaturizationConfig={
        "ForecastFrequency": ForecastFrequency, 
        "Featurizations": [
            {
                "AttributeName": "target_value", 
                "FeaturizationPipeline": [
                    {
                        "FeaturizationMethodName": "filling", 
                        "FeaturizationMethodParameters": 
                                {
                                    "frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"
                                }
                              }
                            ]
                          }
                        ]
    }
 )





You can check the training status in the Forecast [console](https://console.aws.amazon.com/forecast/home?region=us-east-1#landing).

Also, you can run the following cell, cause it includes the await condition.

## Examining the Predictors

Once each of the Predictors is in an `Active` state, you can get metrics about it to better understand its accuracy and behavior. These are computed based on the hold out periods we defined when building the Predictor. The metrics are meant to guide our decisions when we use a particular Predictor to generate a forecast.

### Prophet

Here we are going to look to see the metrics from this Predictor like the earlier sessions, we will now add the related data metrics to the table from the previous notebook as well.

In [ ]:
from IPython.display import display, HTML

# Await till training completion
while True:
    modelTrainStatus = forecast.describe_predictor(
        PredictorArn=prophet_create_predictor_response["PredictorArn"]
    )['Status']
    print(modelTrainStatus)
    if modelTrainStatus != 'ACTIVE' and modelTrainStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

# Prophet Metrics
prophet_with_related_data_arn = prophet_create_predictor_response['PredictorArn']
prophet_with_related_data_metrics = forecast.get_accuracy_metrics(PredictorArn=prophet_with_related_data_arn)

prophet_with_related_data_summary_metrics = prophet_with_related_data_metrics["PredictorEvaluationResults"][0]["TestWindows"][0]


prophet_display_data = {
    "RMSE": [
        prophet_summary_metrics["Metrics"]["RMSE"],
        prophet_with_related_data_summary_metrics["Metrics"]["RMSE"]
    ],
    "10%" : [
        prophet_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        prophet_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "50%" : [
        prophet_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        prophet_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"]
    ],
    "90%" : [
        prophet_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        prophet_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"]
    ]
}
prophet_display_data_frame = pd.DataFrame(prophet_display_data, ["Prophet", "Prophet+Related Data"])

display(prophet_display_data_frame)

In comparison with previous results, we haven't got a significant increase in the accuracy. In some cases it even could be a step back.

### DeepAR+

Same as Prophet, now you should look at the metrics from it.

In [ ]:
# Await till training completion
while True:
    modelTrainStatus = forecast.describe_predictor(
        PredictorArn=deeparp_create_predictor_response["PredictorArn"]
    )['Status']
    print(modelTrainStatus)
    if modelTrainStatus != 'ACTIVE' and modelTrainStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

# DeepAR+ Metrics
deeparp_with_related_data_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_with_related_data_metrics = forecast.get_accuracy_metrics(PredictorArn=deeparp_with_related_data_arn)

deeparp_with_related_data_summary_metrics = deeparp_with_related_data_metrics["PredictorEvaluationResults"][0]["TestWindows"][0]

deeparp_display_data = {
    "RMSE": [
        deepar_summary_metrics["Metrics"]["RMSE"],
        deeparp_with_related_data_summary_metrics["Metrics"]["RMSE"]
    ],
    "10%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "50%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"]
    ],
    "90%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"]
    ]
}
deeparp_display_data_frame = pd.DataFrame(deeparp_display_data, ["DeepAR+", "DeepAR+ + Related Data"])

display(deeparp_display_data_frame)

As we can see from the resultant metrics, the DeepAR+ with related data shows the best results for 10% and 50% quantiles.

Additional work would need to be kicked off from here to determine the specific impact of these figures and how they compare to the existing Forecasting approaches performed by your customer.

## AutoML Example

Not sure which algorithm does fit best for your data? Allow the forecast to evaluate which one does.

In [ ]:
# AutoML Specifics
automl_predictorName= project+'_automl_test'

# Build AutoML:
automl_create_predictor_response=forecast.create_predictor(
    PredictorName=automl_predictorName, 
    ForecastHorizon=forecastHorizon,
    PerformAutoML= True,
    PerformHPO=False,
    EvaluationParameters= {
        "NumberOfBacktestWindows": NumberOfBacktestWindows, 
        "BackTestWindowOffset": BackTestWindowOffset
    }, 
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn, 
        "SupplementaryFeatures": [ 
                     { 
                        "Name": "holiday",
                        "Value": "US"
                     }
                  ]
    },
    FeaturizationConfig={
        "ForecastFrequency": ForecastFrequency, 
        "Featurizations": [
            {
                "AttributeName": "target_value", 
                "FeaturizationPipeline": [
                    {
                        "FeaturizationMethodName": "filling", 
                        "FeaturizationMethodParameters": 
                                {
                                    "frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"
                                }
                              }
                            ]
                          }
                        ]
    }
 )

In [ ]:
# Await till training completion
while True:
    modelTrainStatus = forecast.describe_predictor(
        PredictorArn=automl_create_predictor_response["PredictorArn"]
    )['Status']
    print(modelTrainStatus)
    if modelTrainStatus != 'ACTIVE' and modelTrainStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

In [ ]:
# AutoML
automl_arn = automl_create_predictor_response['PredictorArn']
automl_metrics = forecast.get_accuracy_metrics(PredictorArn=deeparp_arn)

automl_summary_metrics = automl_metrics["PredictorEvaluationResults"][0]["TestWindows"][0]

automl_display_data = {
    "RMSE": [
        deepar_summary_metrics["Metrics"]["RMSE"],
        deeparp_with_related_data_summary_metrics["Metrics"]["RMSE"],
        automl_summary_metrics["Metrics"]["RMSE"]
    ],
    "10%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        automl_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "50%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        automl_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "90%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        automl_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ]
}
automl_display_data_frame = pd.DataFrame(automl_display_data, ["DeepAR+", "DeepAR+ + Related Data", "AutoML"])

display(automl_display_data_frame)

## DeepAR+ HPO Auto-Tuning

Depending on the hyperparameters model can produce different metrics. Manual tuning of such parameters is routine labor, so for the DeepAR+ Forecast provides an ability to automatically test different hyperparameters and choose the best one. 

In [ ]:
# DeepAR+ Specifics
deeparp_hpo_predictorName= project+'_deep_hpo'

# Build DeepAR+:
deeparp_hpo_create_predictor_response=forecast.create_predictor(
    PredictorName=deeparp_hpo_predictorName, 
    AlgorithmArn=deepAR_Plus_algorithmArn,
    ForecastHorizon=forecastHorizon,
    PerformAutoML= False,
    PerformHPO=True,
    EvaluationParameters= {
        "NumberOfBacktestWindows": NumberOfBacktestWindows, 
        "BackTestWindowOffset": BackTestWindowOffset
    }, 
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn, 
        "SupplementaryFeatures": [ 
                     { 
                        "Name": "holiday",
                        "Value": "US"
                     }
                  ]
    },
    FeaturizationConfig={
        "ForecastFrequency": ForecastFrequency, 
        "Featurizations": [
            {
                "AttributeName": "target_value", 
                "FeaturizationPipeline": [
                    {
                        "FeaturizationMethodName": "filling", 
                        "FeaturizationMethodParameters": 
                                {
                                    "frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"
                                }
                              }
                            ]
                          }
                        ]
    }
)

In [ ]:
# Await
while True:
    modelTrainStatus = forecast.describe_predictor(
        PredictorArn=deeparp_hpo_create_predictor_response["PredictorArn"]
    )['Status']
    print(modelTrainStatus)
    if modelTrainStatus != 'ACTIVE' and modelTrainStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break
        
# Metrics        
        
deeparp_hpo_arn = deeparp_hpo_create_predictor_response['PredictorArn']
deeparp_hpo_metrics = forecast.get_accuracy_metrics(PredictorArn=deeparp_hpo_arn)

deeparp_hpo_summary_metrics = deeparp_hpo_metrics["PredictorEvaluationResults"][0]["TestWindows"][0]

deeparp_hpo_display_data = {
    "RMSE": [
        deepar_summary_metrics["Metrics"]["RMSE"],
        deeparp_with_related_data_summary_metrics["Metrics"]["RMSE"],
        deeparp_hpo_summary_metrics["Metrics"]["RMSE"]
    ],
    "10%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"],
        deeparp_hpo_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "50%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][1]["LossValue"],
        deeparp_hpo_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ],
    "90%" : [
        deepar_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        deeparp_with_related_data_summary_metrics["Metrics"]["WeightedQuantileLosses"][0]["LossValue"],
        deeparp_hpo_summary_metrics["Metrics"]["WeightedQuantileLosses"][2]["LossValue"]
    ]
}
deeparp_hpo_display_data_frame = pd.DataFrame(deeparp_hpo_display_data, ["DeepAR+", "DeepAR+ + Related Data", "DeepAR+ +Related Data +HPO"])

display(deeparp_hpo_display_data_frame)        

As we can see, we got even better results for the DeepAR+ HPO job. The drawbacks:
- More time to train the model
- HPO could produce a more efficient result than the no-using HPO, but it may not provide the best possible outcome. Choosing subsets of the hyperparameters is a non-trivial task, at least because there is an infinite number of them.


> As a general best practice, you have to provide the `HPOConfig` parameter to the HPO Auto-Tuning predictor to narrow down the optimization field.